# Python Scraping

- 도서관 정보나루 
- 웹 스크래핑을 통해 원하는 도서관의 대출 데이터를 다운로드 하기

- https://dsc-sookmyung.tistory.com/85


In [7]:
# 로그 메시지 숨기기
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [15]:
import requests
from bs4 import BeautifulSoup

url = "https://www.data4library.kr/openDataL"
response = requests.get(url, verify = False) # SSLError

In [16]:
# SSLError 
# https://blog.naver.com/PostView.nhn?blogId=dsz08082&logNo=222093530720&parentCategoryNo=&categoryNo=55&viewDate=&isShowPopularPosts=true&from=search

In [18]:
html = response.text

In [19]:
soup = BeautifulSoup(html, "html5lib")

In [24]:
# CSS 셀렉터 : id 셀렉터로 검색 칸 select
tags = soup.select("#srchText") 
tags

[<input id="srchText" name="srchText" placeholder="Search" title="검색" type="text" value=""/>]

In [27]:
tag= tags[0]
tag.text

''

In [43]:
# 다중 셀렉터로 text select 
tags = soup.select(".link_td a ") 
# text에서 도서관명만 추출
lib_name = tags[0].text.split(' ')[0] 

# 특수문자 제거
# lib_name = ''.join(filter(str.isalnum, tags[0].text))

# Python Selenium 

- https://sites.google.com/a/chromium.org/chromedriver/downloads 

1. 
- https://www.data4library.kr/openDataL
- ~~검색 입력 칸 클릭 > 검색 도서관명 입력 > 검색 버튼 클릭~~
- ~~검색 결과 첫번째 도서관 클릭 > 첫번째 엑셀버튼 클릭~~

2.
- https://www.data4library.kr/libDataL 참여도서관 목록 
- 검색 입력 칸 클릭 > 검색 도서관명 입력 > 검색 버튼 클릭 
- 검색 결과 첫번째 도서관 클릭 > 도서관 코드 크롤링 
- 도서관명 + 도서관 코드 저장

In [64]:
import selenium
from selenium import webdriver

options = webdriver.ChromeOptions()
options.binary_location = 'C:/Program Files/Google/Chrome/Application/chrome.exe'

browser = webdriver.Chrome('chromedriver.exe', options = options)

In [72]:
URL = "https://www.data4library.kr/libDataL"

driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url=URL)

In [73]:
# 검색 창에 도서관명 입력
search_box = driver.find_element_by_xpath('//*[@id="srchText"]')
search_box.send_keys('한밭도서관')

In [75]:
# 검색 버튼 클릭
search_btn = driver.find_element_by_xpath('//*[@id="pagef"]/div[3]/div[1]/div/a')
search_btn.click()

In [77]:
# 첫번째 도서관 클릭
driver.find_element_by_xpath('//*[@id="pagef"]/div[4]/table/tbody/tr/td[2]/a').click()

In [82]:
lib_code_list = []

In [84]:
lib_code = driver.find_element_by_xpath('//*[@id="sb-site"]/section/div[5]/div/table/tbody/tr[1]/td[2]')
lib_code_list.append(lib_code.text)

In [85]:
lib_code_list

['125003']

In [ ]:
# 도서관 API 

- 도서관 정보나루
- https://www.data4library.kr/apiUtilization

- 도서관별 통합정보 -> 도서관별 장서/대출 데이터 조회

- 도서관정보나루_API_Manual.hwp 를 참고하여 작성

- 도서관명 / 

In [2]:
import urllib.request
import json
import pandas as pd
from datetime import datetime,timedelta
import requests
import xmltodict


key = '9e70f846d07f0a8577bf0eec924305c22cec843db920f6ed5c3c0ca4cc871b03'
startDt = '2019-01-01'
endDt = '2021-10-31'

libCode = '125003'

# Call Back URL + 인증키 + 날짜

url = 'http://data4library.kr/api/itemSrch?libCode=' + libCode + '&authKey=' + key + '&startDt=' + startDt + '&endDt=' + endDt + '&pageNo=1&pageSize=10'

url

'http://data4library.kr/api/itemSrch?libCode=125003&authKey=9e70f846d07f0a8577bf0eec924305c22cec843db920f6ed5c3c0ca4cc871b03&startDt=2019-01-01&endDt=2021-10-31&pageNo=1&pageSize=10'

In [3]:
req = requests.get(url).content

xmlObject = xmltodict.parse(req)
xmlObject.

OrderedDict([('response',
              OrderedDict([('request',
                            OrderedDict([('libCode', '125003'),
                                         ('startDt', '2019-01-01'),
                                         ('endDt', '2021-10-31'),
                                         ('pageNo', '1'),
                                         ('pageSize', '10')])),
                           ('libNm', '한밭도서관'),
                           ('pageNo', '1'),
                           ('pageSize', '10'),
                           ('numFound', '53767'),
                           ('resultNum', '10'),
                           ('docs',
                            OrderedDict([('doc',
                                          [OrderedDict([('bookname',
                                                         '악마판사 : 오리지널 대본집 . 2'),
                                                        ('authors', '문유석'),
                                                        ('publisher'

In [6]:
# dict_data = xmlObject['response']['libs']['lib'][0]['loanByhours']['result']
dict_data = xmlObject['response']['docs']['doc'][0]

df_conf = pd.DataFrame(dict_data)

In [7]:
df_conf

,bookname,authors,publisher,publication_year,isbn13,set_isbn13,bookImageURL,addition_symbol,vol,class_no,class_nm,callNumbers,reg_date
callNumber,산다는 건 뭘까?,채인선,미세기,2021,9788980715121,None,https://image.aladin.co.kr/product/27858/76/co...,None,None,126.5,"철학 > 인식론, 인과론, 인간학 >","{'separate_shelf_code': '아', 'separate_shelf_n...",2021-10-27
